In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
# VGG16 모델 불러오기 (사전 학습된 가중치 사용, 마지막 분류 레이어 제외)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

In [7]:
# 경로 설정
train_dir = 'C:/Users/intel/Downloads/trainset(300x300)'
test_dir = 'C:/Users/intel/Downloads/testset(300x300)'

# ImageDataGenerator 설정 (데이터 증강 없이 단순 스케일링만 적용)
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# 훈련용 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    subset='training',      # 훈련용 데이터
    shuffle=True
)

# 검증용 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    subset='validation',    # 검증용 데이터
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    shuffle=False             # 테스트 데이터는 셔플하지 않음
)

Found 38280 images belonging to 100 classes.
Found 4211 images belonging to 100 classes.
Found 10587 images belonging to 100 classes.


In [9]:
# 간단한 모델 생성
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(100, activation='softmax')  # 100개의 클래스로 분류
])

# 모든 층을 학습 가능하게 설정
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
# 모델 학습
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
1197/1197 [==============================] - 405s 329ms/step - loss: 3.9122 - accuracy: 0.0840 - val_loss: 3.4335 - val_accuracy: 0.1221
Epoch 2/10
1197/1197 [==============================] - 382s 319ms/step - loss: 3.2978 - accuracy: 0.1410 - val_loss: 3.3172 - val_accuracy: 0.1332
Epoch 3/10
1197/1197 [==============================] - 384s 320ms/step - loss: 3.1379 - accuracy: 0.1668 - val_loss: 3.3711 - val_accuracy: 0.1282
Epoch 4/10
1197/1197 [==============================] - 387s 323ms/step - loss: 3.0315 - accuracy: 0.1823 - val_loss: 3.2940 - val_accuracy: 0.1460
Epoch 5/10
1197/1197 [==============================] - 381s 318ms/step - loss: 2.9370 - accuracy: 0.1994 - val_loss: 3.2880 - val_accuracy: 0.1544
Epoch 6/10
1197/1197 [==============================] - 378s 316ms/step - loss: 2.8721 - accuracy: 0.2120 - val_loss: 3.3154 - val_accuracy: 0.1579
Epoch 7/10
1197/1197 [==============================] - 380s 318ms/step - loss: 2.8080 - accuracy: 0.2196 - val_